# Activation du PWM sur la broche 32 du port d'extension GPIO sur le jetson nano

### Solution simple (mais qui ne fonctionne pas...) :

Remarque : il faut activer le PWM de la broche 32 et enregistrer la configuration (pas besoin de le refaire à chaque fois) :  
=> sudo python /opt/nvidia/jetson-io/jetson-io.py

*Remarque : **Cette méthode semble ne pas fonctionner**...  

Voici donc une autre méthode avec écriture dans les registres internes du Jetson Nano

### Solution en configurant les registres interne du SoC

#### Broches du GPIO et fonctions associées

Les broches pouvant fonctioner en PWM sur le jetson nano sont :  
    - Broche 32 (BCM n°12) : PWM0  
    - Broche 33 (BCM n°13) : PWM2  

Par exemple, la pin32 est BCM=12 alors que pin12 est BCM=18.


<center><img src="https://github.com/AlexandreBourrieau/FICHIERS/blob/main/Olympidades2024/JetsonNanoBoard.jpg?raw=true" width=700></center>

#### Activer le PWM0 sur la broche 32 du Jetson Nano par écriture directe dans le registre interne

Pinmux spreadsheet : https://developer.nvidia.com/jetson-nano-pinmux  
Technical reference manual: https://developer.nvidia.com/embedded/dlc/tegra-x1-technical-reference-manual  
Jetson Nano Module Datasheet : https://developer.nvidia.com/embedded/dlc/jetson-nano-datasheet  
Jetson Nano System-on-Module Data Sheet : https://developer.nvidia.com/embedded/dlc/jetson-nano-system-module-datasheet

Dans le spreadsheet, on voit que le PWM0 de la broche 32 a pour fonction auxilaire LCD_BL_PWM, ce qui correspond au registe  **PINMUX_AUX_LCD_BL_PWM_0**

<center><img src="https://github.com/AlexandreBourrieau/FICHIERS/blob/main/Olympidades2024/Spreadsheet.jpg?raw=true" width=1024></center>

L'adresse de base du registre PINMUX_X est 0x70000000:

<center><img src="https://github.com/AlexandreBourrieau/FICHIERS/blob/main/Olympidades2024/AddrMapPinmux.jpg?raw=true" width=600></center>

Le registre **PINMUX_AUX_LCD_BL_PWM_0** se trouve à l'offset 0x31fc (adresse de base 0x70000000). On va le configurer de la manière suivante:
 - PM = 1 (Activation de la fonction PWM0) => Bit 1:0 = 01
 - PUPD = 2 (Activation de la PULL_UP) => Bit3:2 = 10
 - TRISTATE = 0
 - PARK = 0
 - E_INPUT = 0 (Désactive le mode input receiver)
 - Tout le reste par défaut

=> La valeur a écrire est donc : %0000000001001 = 0x05

<center><img src="https://github.com/AlexandreBourrieau/FICHIERS/blob/main/Olympidades2024/RegistrePWM0.jpg?raw=true" width=500></center>

Ensuite, comme par défaut la broche 32 est configurée en mode GPIO, il faut la configurer en mode SFIO (Special Function I/O) pour utiliser la foncion spéciale PWM de cette broche.

La broche 32 du port GPIO (dénommée GPIO07), est la broche n°0 du port V (22) : (il y a en tout 32 ports : A,B,...,Z, AA, BB, CC, DD, EE, FF et chaque port possède 8 broches)

<center><img src="https://github.com/AlexandreBourrieau/FICHIERS/blob/main/Olympidades2024/JetsonPortV.jpg?raw=true" width=400></center>

Dans le cicuit du jetson nano (SoC), les registres permettant de cibler les broches sont organisés en 8 **GPIO Controller**, et chaque Controller possède 4 sous-groupes (ce qui correspond aux 32 ports précédents). La broche 32 est donc dans le GPIO Controller n°(22/4 + 1) (les GPIO Controller sont numérotés à partir de 1) = 6,5 soit le GPIO Controller n°6, à la 2ème position.  

On peut y lire l'offset pour accéder au registre GPIO_CNF_1 qui permettra de configurer cette broche en mode SFIO (Special Function I/O - mode PWM) :

<center><img src="https://github.com/AlexandreBourrieau/FICHIERS/blob/main/Olympidades2024/GPIOControllerOffset.jpg?raw=true" width=600></center>

Le registre GPIO_CNF_1 est structuré de la manière suivante :

<center><img src="https://github.com/AlexandreBourrieau/FICHIERS/blob/main/Olympidades2024/GPIO_CNF.jpg?raw=true" width=600></center>

Pour activer le mode SFIO sur la broche 0, on doit donc mettre le bit 0 à 0.

L'adresse de base des GPIO est 0x6000d000. Puisque l'offset du resgistre est 504, il faudra donc écrire la valeur à l'adresse 0x6000d504, qui correspond bien à une adresse appartenant au GPIO-6 :

<center><img src="https://github.com/AlexandreBourrieau/FICHIERS/blob/main/Olympidades2024/AdresseGPIOg62.jpg?raw=true" width=600></center>

Pour activer le PWM0 sur la broche 32:

=> sudo busybox devmem 0x700031fc 32 0x05 $\;\;\;\;\;\;$     *// Ecrit la valeur 0x05 (0000 0101) sur 32bits à l'adresse 0x700031fc*  
=> sudo busybox devmem 0x6000d504 32 0x00  $\;\;\;\;\;\;$    *// Ecrit la valeur 0x00 (0000 0000) sur 32bits à l'adresse 0x6000d504*

#### Activer les PWM au reboot

=> sudo nano /etc/systemd/system/rc-local.service

Placer le texte suivant dans le fichier:

[Unit]  
 Description=/etc/rc.local Compatibility  
 ConditionPathExists=/etc/rc.local  

[Service]  
 Type=forking  
 ExecStart=/etc/rc.local start  
 TimeoutSec=0  
 StandardOutput=tty  
 RemainAfterExit=yes  
 SysVStartPriority=99  

[Install]  
 WantedBy=multi-user.target  

=> sudo nano /etc/rc.local

Placer le texte suivant dans le fichier:

#!/bin/bash

sudo busybox devmem 0x700031fc 32 0x05  
sudo busybox devmem 0x6000d504 32 0x00  

=> sudo chmod +x /etc/rc.local

=> sudo systemctl enable rc-local